<a href="https://colab.research.google.com/github/luanakwon/ML_Pytorch_pre2022/blob/main/GAN_PR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First attempt of GAN model using mnist dataset.  
(probably failed)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optimizer
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# HyperParameters

z_len = 50
middle_size = 400
batch_size = 128
learning_rate = 0.0001
num_epoch = 60

In [ ]:
'''
mnist dataset download
생성자와 구별자의 경쟁이기 때문에 testset은 받지 않음
'''

mnist_train = dset.MNIST("./",train=True, transform=transforms.ToTensor(),target_transform=None, download=True)

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
'''
기본 생성자. cnn을 써야되는데, 어떻게 쓸 수 있는지 모르겠어서 우선 기본으로 둠
맨 처음 z를 100에서 2*2*64로 프로젝션 하는 부분은 학습하지 않음(requires_grad = False)
이후 conv레이어만 학습하기 위해 임시로 layer_conv를 선형모델로 둠
'''
class Generator(nn.Module):
  def __init__(self):
    super(Generator, self).__init__()
    self.layer_lin = nn.Linear(z_len,2*2*64)
    self.layer_conv = nn.Linear(1,1) # 

  def forward(self,x):
    out = self.layer_lin(x)
    out = out.view(batch_size,64,2,2)
    out = self.layer_conv(out)
    #print(out.shape)
    return  out

gen_layer_conv = nn.Sequential(
        nn.ConvTranspose2d(64,32,2,2),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.ConvTranspose2d(32,16,4,1),#(i-1)*s + k
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.ConvTranspose2d(16,8,2,2),
        nn.ReLU(),
        nn.BatchNorm2d(8),
        nn.ConvTranspose2d(8,1,2,2),
        nn.Sigmoid()
    )


nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.ConvTranspose2d(64,32,7),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.ConvTranspose2d(32,16,7),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.ConvTranspose2d(16,8,8,),
        nn.Sigmoid()

In [ ]:
'''
구별자. 기본으로는 성능이 떨어진다고 해서 합성곱레이어 추가
'''

class Discriminator(nn.Module):
  def __init__(self):
    super(Discriminator, self).__init__()
    self.layer0 = nn.Sequential(
      nn.Linear(784,400),
      nn.LeakyReLU(),
      nn.BatchNorm1d(400),
      nn.Linear(400,150),
      nn.LeakyReLU(),
      nn.BatchNorm1d(150),
      nn.Linear(150,1),
      nn.Sigmoid()
    )
    self.layer_conv = nn.Sequential(
      nn.Conv2d(1,16,4,2),
      nn.LeakyReLU(),
      nn.BatchNorm2d(16),
      nn.Conv2d(16,32,5,2),
      nn.LeakyReLU(),
      nn.BatchNorm2d(32),
      nn.Conv2d(32,16,3,2),
      nn.LeakyReLU(),
      nn.BatchNorm2d(16),
      nn.Conv2d(16,1,2),
      nn.Sigmoid()
    )
  def forward(self, x):
    out = self.layer_conv(x)
    out = out.view(batch_size,-1)
    return out

In [ ]:
'''
device에 올리기. DataParellel을 쓰니까 dis_loss할때 cpu에 텐서가 있대서 이렇게 바꿈. 느려도 작동함
'''
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

generator = Generator().to(device)
discriminator = Discriminator().to(device)


#generator = nn.DataParallel(Generator())
#discriminator = nn.DataParallel(Discriminator())

In [ ]:
for param in generator.parameters():
  param.requires_grad = False
generator.layer_conv = gen_layer_conv.to(device)

In [ ]:
print(generator)

Generator(
  (layer_lin): Linear(in_features=50, out_features=256, bias=True)
  (layer_conv): Sequential(
    (0): ConvTranspose2d(64, 32, kernel_size=(2, 2), stride=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(16, 8, kernel_size=(2, 2), stride=(2, 2))
    (7): ReLU()
    (8): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(8, 1, kernel_size=(2, 2), stride=(2, 2))
    (10): Sigmoid()
  )
)


In [ ]:
loss_func = nn.BCELoss()
gen_optim = optimizer.Adam(generator.parameters(),lr=learning_rate*1, betas=(0.5,0.999))
dis_optim = optimizer.Adam(discriminator.parameters(),lr=learning_rate,betas=(0.5,0.999))

ones_label = torch.ones(batch_size,1).to(device)
zeros_label = torch.zeros(batch_size,1).to(device)

In [ ]:
'''
학습과정 lr을 조금 바꿔봤는데도, 얼마 안가서 gen_loss = 0.25, dis_loss는 0.5로 수렴해버림. 왜?
'''
for i in range(num_epoch):
  for j, (image,label) in enumerate(train_loader):
    #discriminator learning
    
    dis_optim.zero_grad()

    z = init.normal_(torch.Tensor(batch_size,z_len),std=1).to(device)
    gen_fake = generator.forward(z)
    dis_fake = discriminator.forward(gen_fake)

    image = image.to(device)
    dis_real = discriminator.forward(image)
    #dis_loss = torch.mean(loss_func(dis_fake, zeros_label)) + torch.mean(loss_func(dis_real, ones_label))
    dis_loss = torch.mean(dis_fake) + torch.mean(1- dis_real)
    if j%1000 == 0:
      #print(torch.sum(loss_func(dis_fake, zeros_label)).item(), torch.sum(loss_func(dis_real, ones_label)).item(),end=' ')
      print("%d dis_fake : %0.4f   dis_real : %0.4f    " % (i, torch.sum(dis_fake).item()/256 ,torch.sum(dis_real).item()/256), end='')
    dis_loss.backward()
    dis_optim.step()
      
    for k in range(2):
      #generator learning
      gen_optim.zero_grad()

      z = init.normal_(torch.Tensor(batch_size,z_len),std=1).to(device)
      gen_fake = generator.forward(z)
      dis_fake = discriminator.forward(gen_fake)

      #gen_loss = torch.mean(loss_func(dis_fake,ones_label))
      gen_loss = torch.mean(1 - dis_fake)
      gen_loss.backward()
      gen_optim.step()


    if j%1000 == 0:
      print("gen_fake: %0.4f" %(torch.sum(dis_fake).item()/256))

0 dis_fake : 0.2518   dis_real : 0.2625    gen_fake: 0.2564
1 dis_fake : 0.2425   dis_real : 0.4855    gen_fake: 0.2512
2 dis_fake : 0.1948   dis_real : 0.4889    gen_fake: 0.1995
3 dis_fake : 0.1431   dis_real : 0.4903    gen_fake: 0.1349


KeyboardInterrupt: ignored

In [ ]:
save_path = './drive/MyDrive/omok/'
torch.save(generator.state_dict(),save_path + 'GAN_generator2.pth')
torch.save(discriminator.state_dict(),save_path + 'GAN_discriminator2.pth')

In [ ]:
import numpy as np

st = 55
with torch.no_grad():
  z = init.normal_(torch.Tensor(batch_size, z_len),std=1).to(device)
  gen_fake = generator.forward(z)
  dis_fake = discriminator.forward(gen_fake)
  
  maxi = np.argmax(dis_fake.cpu().detach().numpy())
  mini = np.argmin(dis_fake.cpu().detach().numpy())
  
  image = []
  image.append(torch.reshape(gen_fake[mini],(28,28)).cpu().detach().numpy())
  image.append(torch.reshape(gen_fake[maxi],(28,28)).cpu().detach().numpy())

  print(dis_fake[mini], dis_fake[maxi])
  plt.figure(figsize=(20,30))
  for i in range(2):
      plt.subplot(1,5,i+1)
      plt.imshow(image[i], cmap='gray')

In [ ]:
with torch.no_grad():
  it = iter(train_loader)
  image, label = next(it)
  x = image.to(device)
  dis_real = discriminator.forward(x)

  maxi = np.argmax(dis_real.cpu().detach().numpy())
  mini = np.argmin(dis_real.cpu().detach().numpy())
  print(mini, maxi)
  image = []
  image.append(torch.reshape(x[mini],(28,28)).cpu().detach().numpy())
  image.append(torch.reshape(x[maxi],(28,28)).cpu().detach().numpy())

  print(dis_real[mini].item(), dis_real[maxi].item())
  plt.figure(figsize=(20,30))
  for i in range(2):
      plt.subplot(1,5,i+1)
      plt.imshow(image[i], cmap='gray')

In [ ]:
it = iter(train_loader)
image,label = next(it)
sample = torch.reshape(image[0],(28,28)).cpu().detach().numpy()
fig = plt.figure
plt.imshow(sample, cmap='gray')
plt.show()

In [ ]:
256 * 234

In [ ]:
'''
ascii image 생성 함수. 이미지 처리도 배워야겠다..
'''
#import numpy as np

def print_image(image):
  im = np.array(image)
  color = [' ', '.', ':', '|', '@', '@']

  im *= (len(color) - 1)

  for r in im:
    for e in r:
      print(color[int(e)],end=' ')
    print()

In [ ]:
'''
그리기 테스트. 학습이 안되서 아직 의미 없음
'''
test_size = 1

generator.eval()
discriminator.eval()

with torch.no_grad():
  for i in range(test_size):
    z = init.normal_(torch.Tensor(batch_size,z_len),std=0.1).to(device)
    gen_fake = generator.forward(z)
    dis_fake = discriminator.forward(gen_fake)

    print_image(gen_fake.cpu()[0,0])
    print(dis_fake[0])

In [ ]:
import numpy as np


with torch.no_grad():
  z = init.normal_(torch.Tensor(batch_size, z_len),std=1).to(device)
  gen_fake = generator.forward(z)
  dis_fake = discriminator.forward(gen_fake)
  
  maxi = np.argmax(dis_fake.cpu().detach().numpy())
  mini = np.argmin(dis_fake.cpu().detach().numpy())
  
  image = []
  image.append(np.transpose(torch.reshape(gen_fake[mini],(3,image_size,image_size)).cpu().detach().numpy(), (1,2,0))
  image.append(np.transpose(torch.reshape(gen_fake[maxi],(3,image_size,image_size)).cpu().detach().numpy(), (1,2,0))

  print(dis_fake[mini], dis_fake[maxi])
  plt.figure(figsize=(20,30))
  for i in range(2):
      plt.subplot(1,5,i+1)
      plt.imshow(image[i])